# Parte A

## Sistema de paragem ABS

In [36]:
from z3 import *
import sys, os

In [37]:
Modes, (Start, Free, Stopping, Blocked, Stopped) = EnumSort('Modes',['Start', 'Free', 'Stopping', 'Blocked', 'Stopped'])
Events, (start, goStopping, goFree, goBlocked, goStopped , alive , dead) =  \
    EnumSort('Events',['start', 'goStopping', 'goFree', 'goBlocked', 'goStopped' , 'alive' , 'dead'])
    


In [38]:
N=10
vv   = RealVector('vv', N+1) #velocidade do veiculo
vr   = RealVector('vr', N+1) #velocidade das rodas
temp = RealVector('t', N+1) #velocidade das rodas
modes  = AstVector() ; modes.resize(N+1)
events = AstVector() ; events.resize(N+1)

for k in range(N+1):
    modes[k] = Const('md_'+str(k), Modes)

for k in range(N+1):
    events[k] = Const('ev_'+str(k), Events)

def S(k):
    return (vv[k],vr[k],temp[k],modes[k])

def E(k):
    return events[k]

In [39]:
def init(S):
    (V,R,T,M) = S 
    return And(R <= V, M == Start)

In [40]:
def inv(S):
    (V,R,T,M) = S 
    return And(Implies( M == Start    , T == 0              ),
               Implies( M == Free     , V >= R              ),
               Implies( M == Stopping , And(R >= 0, V >= R) ),
               Implies( M == Blocked  , And(R == 0, V >= 0) ),
               Implies( M == Stopped  , And(V == R, R == 0) )
             )

In [41]:
def timed(S,S_,E):
    (V,R,T,M) = S ; (V_,R_,T_,M_) = S_ 
    return And( E == alive, 
                M == M_,
                T_ > T,
                Implies( M == Start    , And(V == V_, R == R_)                                        ),
                Implies( M == Free     , And((V_ - V) + Fd*(T_ - T) == 0, (R_ - R) - c*(T_ - T) == 0) ),
                Implies( M == Stopping , And((V_ - V) + a*(T_ - T) == 0, (R_ - R) + b*(T_ - T) == 0)  ),
                Implies( M == Blocked  , And((V_ - V) + Fe*(T_ - T) == 0, R == R_, R == 0)            ),
                Implies( M == Stopped  , And(V == V_, R == R_)                                        )
              ) 

In [ ]:
#-----------------------
def untimed(S,S_,E):
    (V,R,T,M) = S ; (V_,R_,T_,M_) = S_ 
    return And( V == V_,
                R == R_,
                T == T_,
                Implies( M == Init  , And(M_==Fill, T_ == T, E == initialize, Y == 0)),
                Implies( M == Fill  , And(M_ == Stop, T_ == 0, E == goStop, Y == 10)),
                Implies( M == Drain , And(M_ == Start, T_ == 0, E == goStart, Y == 5)),
                Implies( M == Start , And(M_ == Fill, T_ == T, E == goFill, T == 2)),
                Implies( M == Stop  , And(M_ == Drain, T_ == T, E == goDrain, T == 2))
              )

In [ ]:
def blocked(S,S_,E): 
    (Y,T,M) = S ; (Y_,T_,M_) = S_ 
    return  And(E == dead, Y_ == Y, T_ == T, M_ == M)

In [ ]:
def trans(S,S_,E): 
    return Or(blocked(S,S_,E), timed(S,S_,E), untimed(S,S_,E))

# Parte B

## Codificar o algoritmo de Euclides da questao 2 do Trabalho 2

0: $assert\quad a > 0\quad and\quad b > 0$  
1: $while\quad a > 0\quad and\quad b > 0:$  
2: $\hspace{1cm}if\quad b > a$  
3: $\hspace{2cm}b = b - a$  
4: $\hspace{1cm}else:$  
5: $\hspace{2cm}a = a - b$    
6: $pass$ 


## Código em 'comandos anotados'

```
# pré-condição:
assume a > 0  and  b > 0;
x := a;
y := b;

# condição para a entrada no ciclo:
assert INV(x,y);

# condição para cada iteracção: 
havoc x;
havoc y;
assume INV(x,y);


if x = 0:
    # condição para a saída do ciclo (pós-condição)
    assert INV(x,y) and (y = gcd(a,b));
    assume false
else:
    if y > x:
        y := y - x
    else:
        x := x - y
    ;
    # condição para a saída de iteracção:
    assert INV(x,y)
```

## INV:
O nosso invariante será:

In [22]:
def INV(x,y):
    F = x>=0 and y>0 

In [23]:
import random 

def nounce(pref,srt=BoolSort()):
    n = hex(hash(random.random()))[-6:-2]   # 
    return Const(pref + n, srt)

'''vou precisar de um x e de um y
X = nounce('X',Bez)
Y = nounce('Y',Bez)
'''

def spc(f):
    return simplify(B0(B1(B2(f))))

def B0(f):
    A = variavel fresca ;  B = variavel fresca
    F= And(a > 0 , b > 0, f)
    Q= And(substitute(F,(X,A),(Y,B)),X==substitute(a,(X,A),Y==substitute(b,(Y,B))))
    return Implies(Q,INV(X,Y))
                   
def B1(f):
    A = variavel fresca ;  B = variavel fresca
    Q= And(substitute(f,(X,A),(Y,B)),X==A,Y==B)
    return And(Q,INV(X,Y))


def B2(f):
    A = variavel fresca ;  B = variavel fresca
    return ite = And(
                And(f,And(Implies( X == 0 ,INV(X,Y)),False)
                    )
                ,And(f,
                    And( X != 0 , Y > X, And(substitute(f,(Y,B)),Y == substitute(Y-X,(Y,B)))
                       )
                 
                    ,And( X != 0 , Y <= X,And(substitute(f,(X,A)),X == substitute(X-Y,(X,A)))
                        )
                    )
    
                )
            
    
  
'''
# para provar que 'wpc(true)' é tautologia, prova-se que a sua negação é inconsistente
true = BoolVal(True)
refutation = Not(wpc(true))
'''


SyntaxError: invalid syntax (<ipython-input-23-b498ecd8cbc0>, line 16)